Importation des données

In [ ]:
import os
import pandas as pd

# demande à l'utilisateur le chemin vers la BD
# chemin_BD = input("Entrez le chemin vers la base de données: ")
chemin_BD = "./dataset/UCI_HAR_Dataset/UCI HAR Dataset"

# Supprime les espaces en début et fin de chaîne de caractères
chemin_BD=chemin_BD.strip()

# Chemin vers le dossier train
chemin_train = os.path.join(chemin_BD, "train")
# Chemin vers le dossier test
chemin_test = os.path.join(chemin_BD, "test")

# Fonction pour lire un fichier
def lire_fichier(path):                                  
    return pd.read_csv(path, delim_whitespace=True, header=None) # delim_whitespace=True permet de lire les données séparées par des espaces (et non des virgules)

# Lecture (données brutes et annotations) du dossier train
donnees_brutes_train = lire_fichier(os.path.join(chemin_train, "X_train.txt"))
annotations_train = lire_fichier(os.path.join(chemin_train, "y_train.txt"))
y_train=annotations_train.values.ravel()  

# Lecture (données brutes et annotations) du dossier test
donnees_brutes_test = lire_fichier(os.path.join(chemin_test, "X_test.txt"))
annotations_test = lire_fichier(os.path.join(chemin_test, "y_test.txt"))

Visualisation des données

In [ ]:
donnees_brutes_train.head()

In [ ]:
donnees_brutes_train.info()

In [ ]:
donnees_brutes_train.describe()

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
donnees_brutes_train.hist(figsize=(50, 50))
plt.show()

Extraction des caractéristiques (métriques statistiques)

In [ ]:
import numpy as np
from scipy.stats import entropy, mode, skew, kurtosis

def entropie(signal):
    # Histogramme
    histogramme = np.histogram(signal, bins=100)[0]
    # Normalisation
    histogramme = histogramme / np.sum(histogramme)
    # Entropie
    return entropy(histogramme)

def frequence_moyenne(signal):
    # Transformée de Fourier
    frequence_amplitude = np.abs(np.fft.fft(signal))
    freqs=np.fft.fftfreq(len(signal))
    # Fréquence moyenne
    return np.average(freqs, weights=frequence_amplitude)

# Extraction des caractéristiques statistiques (Train)
moyenne_train = np.mean(donnees_brutes_train, axis=1)    # Moyenne
maximum_train = np.max(donnees_brutes_train, axis=1)     # Maximum
minimum_train = np.min(donnees_brutes_train, axis=1)     # Minimum
variance_train = np.var(donnees_brutes_train, axis=1)    # Variance
ecart_type_train = np.std(donnees_brutes_train, axis=1)  # Écart-type 
mediane_train = np.median(donnees_brutes_train, axis=1)  # Médiane
sma_train = np.sum(np.abs(donnees_brutes_train), axis=1) # Somme des valeurs absolues
energie_train = np.sum(donnees_brutes_train**2, axis=1)  # Énergie
iqr_train = np.percentile(donnees_brutes_train, 75, axis=1) - np.percentile(donnees_brutes_train, 25, axis=1) # Écart interquartile
entropie_train = np.apply_along_axis(entropie, 1, donnees_brutes_train) # Entropie
coefficient_corr_train = np.corrcoef(donnees_brutes_train) # Coefficient de corrélation
indices_max_freq_train = np.argmax(np.abs(np.fft.fft(donnees_brutes_train)), axis=1) # Indice de la fréquence maximale
frequences_moyennes_train = np.apply_along_axis(frequence_moyenne, 1, donnees_brutes_train) # Fréquence moyenne
mode_train = mode(donnees_brutes_train, axis=1)[0] # Mode
skew_train = skew(donnees_brutes_train, axis=1) # Skewness
kurtosis_train = kurtosis(donnees_brutes_train, axis=1) # Kurtosis

# Extraction des caractéristiques statistiques (Test)
moyenne_test = np.mean(donnees_brutes_test, axis=1)     
maximum_test = np.max(donnees_brutes_test, axis=1)      
minimum_test = np.min(donnees_brutes_test, axis=1)      
variance_test = np.var(donnees_brutes_test, axis=1)     
ecart_type_test = np.std(donnees_brutes_test, axis=1)  
mediane_test = np.median(donnees_brutes_test, axis=1)
sma_test = np.sum(np.abs(donnees_brutes_test), axis=1)
energie_test = np.sum(donnees_brutes_test**2, axis=1)
iqr_test = np.percentile(donnees_brutes_test, 75, axis=1) - np.percentile(donnees_brutes_test, 25, axis=1)
entropie_test = np.apply_along_axis(entropie, 1, donnees_brutes_test)
coefficient_corr_test = np.corrcoef(donnees_brutes_test)
indices_max_freq_test = np.argmax(np.abs(np.fft.fft(donnees_brutes_test)), axis=1)
frequences_moyennes_test = np.apply_along_axis(frequence_moyenne, 1, donnees_brutes_test)
mode_test = mode(donnees_brutes_test, axis=1)[0]
skew_test = skew(donnees_brutes_test, axis=1)
kurtosis_test = kurtosis(donnees_brutes_test, axis=1)


# Concaténation des caractéristiques statistiques (vecteur)
X_train_features = np.column_stack((moyenne_train, maximum_train, minimum_train, variance_train, ecart_type_train, mediane_train, sma_train, energie_train, iqr_train, entropie_train, indices_max_freq_train, frequences_moyennes_train, mode_train, skew_train, kurtosis_train))
X_test_features = np.column_stack((moyenne_test, maximum_test, minimum_test, variance_test, ecart_type_test, mediane_test, sma_test, energie_test, iqr_test, entropie_test, indices_max_freq_test, frequences_moyennes_test, mode_test, skew_test, kurtosis_test))

# Visualisation des caractéristiques statistiques
pd.DataFrame(X_train_features).hist(figsize=(20, 20))
pd.DataFrame(X_test_features).hist(figsize=(20, 20))

Extraction des caractéristiques (méthodes de description PyTS)

In [ ]:
from pyts.transformation import BagOfPatterns
from pyts.transformation import BOSS

# Méthode BagOfPatterns
bop = BagOfPatterns(window_size=32, word_size=5, sparse=False, numerosity_reduction=True)  
X_train_bop = bop.fit_transform(donnees_brutes_train)
X_test_bop = bop.transform(donnees_brutes_test)


# Méthode BOSS
boss = BOSS(word_size=5, n_bins=5, window_size=24, sparse=False)
X_train_boss = boss.fit_transform(donnees_brutes_train)
X_test_boss = boss.transform(donnees_brutes_test)

# Concaténation des caractéristiques (vecteur)
X_train_Features = np.column_stack((X_train_bop, X_train_boss))
X_test_Features = np.column_stack((X_test_bop, X_test_boss))

Réduction de la dimensionnalité (avec BagOfPatterns pour l'extraction des caractéristiques)

In [ ]:
from sklearn.decomposition import PCA
"""from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.decomposition import TruncatedSVD
from sklearn.decomposition import FastICA"""

# PCA
pca = PCA(n_components=0.98) 
X_train_pca = pca.fit_transform(X_train_Features)
X_test_pca = pca.transform(X_test_Features)

"""
# LDA
n_classes= len(set(y_train))
lda = LDA(n_components=n_classes-1)
X_train_lda = lda.fit_transform(X_train_Features, y_train)
X_test_lda = lda.transform(X_test_Features)

# SVD
svd = TruncatedSVD(n_components=3)
X_train_svd = svd.fit_transform(X_train_Features)
X_test_svd = svd.transform(X_test_Features)

# ICA
ica = FastICA(n_components=n_classes-1) # usually doesn't converge even with 500 iterations
X_train_ica = ica.fit_transform(X_train_Features)
X_test_ica = ica.transform(X_test_Features)
"""

Classification (apprentissage)

In [ ]:
from sklearn import svm
from sklearn.multiclass import OneVsRestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC 

from sklearn.model_selection import GridSearchCV
import joblib

from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
import matplotlib.pyplot as plt

# Create models directory to save them
PROJECT_ROOT_DIR = "."
DATA_PATH = os.path.join(PROJECT_ROOT_DIR, "data")
os.makedirs(DATA_PATH, exist_ok=True)


# Création du modèle SVM
model_svm=svm.SVC()
# Apprentissage du modèle SVM
model_svm.fit(X_train_pca, y_train)
# Prédiction
y_pred_svm=model_svm.predict(X_test_pca)

"""
parameters = {'n_neighbors': range(1, 20)}

knn = KNeighborsClassifier()
clf = GridSearchCV(knn, parameters, cv=5)  # cv=5 indique qu'on utilise une validation croisée à 5 folds
clf.fit(X_train_pca, y_train)

print('Le meilleur nombre de voisins trouvé est :', clf.best_params_['n_neighbors'])

# Création du modèle KNN
model_knn=KNeighborsClassifier(n_neighbors=clf.best_params_['n_neighbors'])
# Apprentissage du modèle KNN
model_knn.fit(X_train_pca, y_train)
# Prédiction
y_pred_knn=model_knn.predict(X_test_pca)
# Sauvegarde du modèle
joblib.dump(model_svm, os.path.join(MODELS_PATH, "knn.joblib"))


# Création du modèle Random Forest
model_rf=RandomForestClassifier(n_estimators=150)
# Apprentissage du modèle Random Forest
model_rf.fit(X_train_pca, y_train)
# Prédiction
y_pred_rf=model_rf.predict(X_test_pca)
# Sauvegarde du modèle
joblib.dump(model_svm, os.path.join(MODELS_PATH, "rf.joblib"))


# Création du modèle Decision Tree
model_dt=DecisionTreeClassifier(max_depth=5)
# Apprentissage du modèle Decision Tree
model_dt.fit(X_train_pca, y_train)
# Prédiction
y_pred_dt=model_dt.predict(X_test_pca)
# Sauvegarde du modèle
joblib.dump(model_svm, os.path.join(MODELS_PATH, "dt.joblib"))


# Création du modèle MLP
model_mlp=MLPClassifier(hidden_layer_sizes=(100, 100, 100), max_iter=500)
# Apprentissage du modèle MLP
model_mlp.fit(X_train_pca, y_train)
# Prédiction
y_pred_mlp=model_mlp.predict(X_test_pca)
# Sauvegarde du modèle
joblib.dump(model_svm, os.path.join(MODELS_PATH, "mlp.joblib"))


# LinearSVC
model_lsvc = LinearSVC()
model_lsvc.fit(X_train_pca, y_train)
y_pred_lsvc = model_lsvc.predict(X_test_pca)
# Sauvegarde du modèle
joblib.dump(model_svm, os.path.join(MODELS_PATH, "lsvc.joblib"))


# GaussianNB
model_gnb = GaussianNB()
model_gnb.fit(X_train_pca, y_train)
y_pred_gnb = model_gnb.predict(X_test_pca)
# Sauvegarde du modèle
joblib.dump(model_svm, os.path.join(MODELS_PATH, "gnb.joblib"))
"""

Classification (évaluation et validation)

In [ ]:
# Taux de reconnaissance (accuracy)
accuracy = accuracy_score(annotations_test, y_pred_svm)
print("Taux de reconnaissance : ", accuracy)

# Rappel, précision et f1-score
report = classification_report(annotations_test, y_pred_svm, output_dict=True, zero_division=1)
df_classification_report = pd.DataFrame(report).transpose()
df_classification_report = df_classification_report.round(3)
print("\nRapport de classification : \n\n", df_classification_report)

# Matrice de confusion
matrice = confusion_matrix(annotations_test, y_pred_svm)
cm_df = pd.DataFrame(matrice)
cm_df.index=cm_df.index+1                                 # Pour commencer à 1 au lieu de 0
cm_df.columns=cm_df.columns+1
print("\nMatrice de confusion :\n\n", cm_df)
print("")

# Calcul du pourcentage
pourcentage = (matrice.astype('float') / matrice.sum(axis=1)[:, np.newaxis]) * 100

# Affichage de la matrice de confusion avec matplotlib
plt.figure(figsize=(4, 4))
plt.imshow(pourcentage, cmap='coolwarm', vmin=0, vmax=100)
plt.colorbar(format='%1.1f%%') 
plt.xlabel('Valeurs prédites')
plt.ylabel('Vraies valeurs')
plt.title('Matrice de confusion')

for (i, j), z in np.ndenumerate(pourcentage):
    if i == j:
        plt.text(j, i, '{:0.1f}%'.format(z), ha='center', va='center', color='black')
    else:
        plt.text(j, i, '{:0.1f}%'.format(z), ha='center', va='center')

plt.xticks(np.arange(matrice.shape[1]), np.arange(1, matrice.shape[1] + 1))
plt.yticks(np.arange(matrice.shape[0]), np.arange(1, matrice.shape[0] + 1))

plt.show()

In [ ]:
# Sauvegarde des données utiles avec joblib

# Sauvegarde des données d'entrainement
joblib.dump([X_train_pca, y_train], os.path.join(DATA_PATH, "train.gz"))

# Sauvegarde des données de test
joblib.dump([X_test_pca, annotations_test], os.path.join(DATA_PATH, "test.gz"))

# Sauvegarde du modèle SVM
joblib.dump(model_svm, os.path.join(DATA_PATH, "model_svm.gz"))

In [ ]:
# Récupération des données de test
[x_test_joblib, y_test_joblib] = joblib.load(os.path.join(DATA_PATH, "test.gz"))

# Récupération du modèle svm
model_svm_joblib = joblib.load(os.path.join(DATA_PATH, "model_svm.gz"))
# Prédiction
y_pred_svm_joblib = model_svm_joblib.predict(x_test_joblib)

In [ ]:
# Taux de reconnaissance (accuracy)
accuracy = accuracy_score(y_test_joblib, y_pred_svm_joblib)
print("Taux de reconnaissance : ", accuracy)

# Rappel, précision et f1-score
report = classification_report(y_test_joblib, y_pred_svm_joblib, output_dict=True, zero_division=1)
df_classification_report = pd.DataFrame(report).transpose()
df_classification_report = df_classification_report.round(3)
print("\nRapport de classification : \n\n", df_classification_report)

# Matrice de confusion
matrice = confusion_matrix(y_test_joblib, y_pred_svm_joblib)
cm_df = pd.DataFrame(matrice)
cm_df.index=cm_df.index+1                                 # Pour commencer à 1 au lieu de 0
cm_df.columns=cm_df.columns+1
print("\nMatrice de confusion :\n\n", cm_df)
print("")

# Calcul du pourcentage
pourcentage = (matrice.astype('float') / matrice.sum(axis=1)[:, np.newaxis]) * 100

# Affichage de la matrice de confusion avec matplotlib
plt.figure(figsize=(4, 4))
plt.imshow(pourcentage, cmap='coolwarm', vmin=0, vmax=100)
plt.colorbar(format='%1.1f%%') 
plt.xlabel('Valeurs prédites')
plt.ylabel('Vraies valeurs')
plt.title('Matrice de confusion')

for (i, j), z in np.ndenumerate(pourcentage):
    if i == j:
        plt.text(j, i, '{:0.1f}%'.format(z), ha='center', va='center', color='black')
    else:
        plt.text(j, i, '{:0.1f}%'.format(z), ha='center', va='center')

plt.xticks(np.arange(matrice.shape[1]), np.arange(1, matrice.shape[1] + 1))
plt.yticks(np.arange(matrice.shape[0]), np.arange(1, matrice.shape[0] + 1))

plt.show()